In [1]:
!pip install wget
!pip install nltk
!pip install multiprocessing
!pip install pymorphy2
!pip install corus

In [2]:
import os
import random
import pymorphy2
import wget
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import multiprocessing
from corus import load_lenta2

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Катерина\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Катерина\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Катерина\AppData\Roaming\nltk_data...


True

# Загрузка набора данных Lenta.ru v1.1+ (news)
Загрузка происходит в папку /data, которая будет создана, если ее еще нет в проекте.

In [4]:
url = 'https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2'
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('data/lenta-ru-news.csv.bz2'):
    wget.download(url, 'data')

In [5]:
path = 'data/lenta-ru-news.csv.bz2'
records = load_lenta2(path)
record_list = list(records)

# Нормализация
Было решено оставить цифры, т.к. данный датасет является новостным, в нем встречаются года и даты, который могут быть важны. Остальные знаки препинания, битые символы было решено убрать с помощью регулярного выражения. Все слова были приведены к нижнему регистру. Для примера были выбраны случайно 1000 записей из общего датасета, чтобы быстрее проверить работоспособность. Ниже будет представлено решение с распараллеливанием для более быстрой работы на больших массивах данных (вызов функции из файла preprocessing.py).

In [32]:
list_text = []
for text in record_list:
    list_text.append(text.text)
test_l = random.sample(list_text, 1000)
print(test_l[0])

В ночь на четверг, 20 марта, в городе Иваново обрушился купол строящегося спортивного комплекса, сообщает Интерфакс со ссылкой на главного инженера компании-подрядчика. По его словам, строителей на объекте во время происшествия не было, сотрудники охраны находились в другом здании, так что жертв и пострадавших нет. Купол здания рухнул под тяжестью мокрого снега, в большом количестве выпавшего в ночь со среды на четверг в Ивановской области. По данным синоптиков, на дорогах города толщина снега сейчас достигает 36 см. Сильнее других от снегопада и сильного, шквалистого ветра пострадали Родниковский, Ивановский, Савинский и Лежневский районы области. Строительство спортивного комплекса должно было завершится к маю этого года. Общая стоимость объекта составляет 260 миллионов рублей. Под куполом на 7,7 тысяч квадратных метров разместятся хоккейное поле, площадка для катания на роликах, сауна, теннисные корты, раздевалки, кафе на 120 мест, спортивные магазины, автодром, площадки для волейбо

In [33]:
filtered_text = [re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s]', '', text.lower()) for text in test_l]
print(filtered_text[0])

в ночь на четверг 20 марта в городе иваново обрушился купол строящегося спортивного комплекса сообщает интерфакс со ссылкой на главного инженера компанииподрядчика по его словам строителей на объекте во время происшествия не было сотрудники охраны находились в другом здании так что жертв и пострадавших нет купол здания рухнул под тяжестью мокрого снега в большом количестве выпавшего в ночь со среды на четверг в ивановской области по данным синоптиков на дорогах города толщина снега сейчас достигает 36 см сильнее других от снегопада и сильного шквалистого ветра пострадали родниковский ивановский савинский и лежневский районы области строительство спортивного комплекса должно было завершится к маю этого года общая стоимость объекта составляет 260 миллионов рублей под куполом на 77 тысяч квадратных метров разместятся хоккейное поле площадка для катания на роликах сауна теннисные корты раздевалки кафе на 120 мест спортивные магазины автодром площадки для волейбола и минифутбола если дворец

# Токенизация
Для токенизации используется word_tokenize из nltk. Разбиение производится по словам.

In [34]:
word_tokens_list = [word_tokenize(text, language='russian') for text in filtered_text]
print(word_tokens_list[0])

['в', 'ночь', 'на', 'четверг', '20', 'марта', 'в', 'городе', 'иваново', 'обрушился', 'купол', 'строящегося', 'спортивного', 'комплекса', 'сообщает', 'интерфакс', 'со', 'ссылкой', 'на', 'главного', 'инженера', 'компанииподрядчика', 'по', 'его', 'словам', 'строителей', 'на', 'объекте', 'во', 'время', 'происшествия', 'не', 'было', 'сотрудники', 'охраны', 'находились', 'в', 'другом', 'здании', 'так', 'что', 'жертв', 'и', 'пострадавших', 'нет', 'купол', 'здания', 'рухнул', 'под', 'тяжестью', 'мокрого', 'снега', 'в', 'большом', 'количестве', 'выпавшего', 'в', 'ночь', 'со', 'среды', 'на', 'четверг', 'в', 'ивановской', 'области', 'по', 'данным', 'синоптиков', 'на', 'дорогах', 'города', 'толщина', 'снега', 'сейчас', 'достигает', '36', 'см', 'сильнее', 'других', 'от', 'снегопада', 'и', 'сильного', 'шквалистого', 'ветра', 'пострадали', 'родниковский', 'ивановский', 'савинский', 'и', 'лежневский', 'районы', 'области', 'строительство', 'спортивного', 'комплекса', 'должно', 'было', 'завершится', 'к'

# Удаление стоп-слов
Для удаления стоп-слов использовалась nltk со списокм слов для русского языка.

In [35]:
stop_words = set(stopwords.words('russian'))
filtered_stop_words_list = []
for text in word_tokens_list:
    filtered_stop_words = [word for word in text if word not in stop_words]
    filtered_stop_words_list.append(filtered_stop_words)
print(filtered_stop_words_list[0])

['ночь', 'четверг', '20', 'марта', 'городе', 'иваново', 'обрушился', 'купол', 'строящегося', 'спортивного', 'комплекса', 'сообщает', 'интерфакс', 'ссылкой', 'главного', 'инженера', 'компанииподрядчика', 'словам', 'строителей', 'объекте', 'время', 'происшествия', 'сотрудники', 'охраны', 'находились', 'другом', 'здании', 'жертв', 'пострадавших', 'купол', 'здания', 'рухнул', 'тяжестью', 'мокрого', 'снега', 'большом', 'количестве', 'выпавшего', 'ночь', 'среды', 'четверг', 'ивановской', 'области', 'данным', 'синоптиков', 'дорогах', 'города', 'толщина', 'снега', 'достигает', '36', 'см', 'сильнее', 'других', 'снегопада', 'сильного', 'шквалистого', 'ветра', 'пострадали', 'родниковский', 'ивановский', 'савинский', 'лежневский', 'районы', 'области', 'строительство', 'спортивного', 'комплекса', 'должно', 'завершится', 'маю', 'года', 'общая', 'стоимость', 'объекта', 'составляет', '260', 'миллионов', 'рублей', 'куполом', '77', 'тысяч', 'квадратных', 'метров', 'разместятся', 'хоккейное', 'поле', 'пл

# Стемминг
Для стемминга используется SnowballStemmer из nltk.

In [36]:
stemmer = SnowballStemmer("russian")
stemmer_list = []
for text in filtered_stop_words_list:
    stemming_txt = [stemmer.stem(word) for word in text]
    stemmer_list.append(stemming_txt)
print(stemmer_list[0])

['ноч', 'четверг', '20', 'март', 'город', 'иванов', 'обруш', 'купол', 'строя', 'спортивн', 'комплекс', 'сообща', 'интерфакс', 'ссылк', 'главн', 'инженер', 'компанииподрядчик', 'слов', 'строител', 'объект', 'врем', 'происшеств', 'сотрудник', 'охра', 'наход', 'друг', 'здан', 'жертв', 'пострада', 'купол', 'здан', 'рухнул', 'тяжест', 'мокр', 'снег', 'больш', 'количеств', 'выпа', 'ноч', 'сред', 'четверг', 'ивановск', 'област', 'дан', 'синоптик', 'дорог', 'город', 'толщин', 'снег', 'достига', '36', 'см', 'сильн', 'друг', 'снегопад', 'сильн', 'шквалист', 'ветр', 'пострада', 'родниковск', 'ивановск', 'савинск', 'лежневск', 'район', 'област', 'строительств', 'спортивн', 'комплекс', 'должн', 'заверш', 'ма', 'год', 'общ', 'стоимост', 'объект', 'составля', '260', 'миллион', 'рубл', 'купол', '77', 'тысяч', 'квадратн', 'метр', 'размест', 'хоккейн', 'пол', 'площадк', 'катан', 'ролик', 'саун', 'теннисн', 'корт', 'раздевалк', 'каф', '120', 'мест', 'спортивн', 'магазин', 'автодр', 'площадк', 'волейбол',

# Лемматизация
Для лемматизации используется морфологический анализатор pymorphy2.

In [37]:
morph = pymorphy2.MorphAnalyzer()
lemmatized_list = []
for text in filtered_stop_words_list:
    lemmatized_text = [morph.normal_forms(word)[0] for word in text]
    lemmatized_list.append(lemmatized_text)
print(lemmatized_list[0])

['ночь', 'четверг', '20', 'март', 'город', 'иваново', 'обрушиться', 'купол', 'строиться', 'спортивный', 'комплекс', 'сообщать', 'интерфакс', 'ссылка', 'главное', 'инженер', 'компанииподрядчик', 'слово', 'строитель', 'объект', 'время', 'происшествие', 'сотрудник', 'охрана', 'находиться', 'друг', 'здание', 'жертва', 'пострадать', 'купол', 'здание', 'рухнуть', 'тяжесть', 'мокрый', 'снег', 'большой', 'количество', 'выпасть', 'ночь', 'среда', 'четверг', 'ивановский', 'область', 'данные', 'синоптик', 'дорога', 'город', 'толщина', 'снег', 'достигать', '36', 'смотреть', 'сильный', 'другой', 'снегопад', 'сильный', 'шквалистый', 'ветер', 'пострадать', 'родниковский', 'ивановский', 'савинский', 'лежневский', 'район', 'область', 'строительство', 'спортивный', 'комплекс', 'должный', 'завершиться', 'маю', 'год', 'общий', 'стоимость', 'объект', 'составлять', '260', 'миллион', 'рубль', 'купол', '77', 'тысяча', 'квадратный', 'метр', 'разместиться', 'хоккейный', 'поле', 'площадка', 'катание', 'ролик', '

Для удобства все вышеописанные действия были объединены в одну функцию preprocessing_text(), в которой используется лемматизация.

In [27]:
morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

In [38]:
def preprocessing_text(text):
    filtered_text = re.sub(r'[^a-zA-Zа-яА-ЯёЁ0-9\s]', '', text)
    word_tokens = word_tokenize(filtered_text.lower(), language='russian')
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return [morph.normal_forms(word)[0] for word in filtered_text]

In [39]:
result_list = [preprocessing_text(text) for text in test_l]
print(result_list[0])

['ночь', 'четверг', '20', 'март', 'город', 'иваново', 'обрушиться', 'купол', 'строиться', 'спортивный', 'комплекс', 'сообщать', 'интерфакс', 'ссылка', 'главное', 'инженер', 'компанииподрядчик', 'слово', 'строитель', 'объект', 'время', 'происшествие', 'сотрудник', 'охрана', 'находиться', 'друг', 'здание', 'жертва', 'пострадать', 'купол', 'здание', 'рухнуть', 'тяжесть', 'мокрый', 'снег', 'большой', 'количество', 'выпасть', 'ночь', 'среда', 'четверг', 'ивановский', 'область', 'данные', 'синоптик', 'дорога', 'город', 'толщина', 'снег', 'достигать', '36', 'смотреть', 'сильный', 'другой', 'снегопад', 'сильный', 'шквалистый', 'ветер', 'пострадать', 'родниковский', 'ивановский', 'савинский', 'лежневский', 'район', 'область', 'строительство', 'спортивный', 'комплекс', 'должный', 'завершиться', 'маю', 'год', 'общий', 'стоимость', 'объект', 'составлять', '260', 'миллион', 'рубль', 'купол', '77', 'тысяча', 'квадратный', 'метр', 'разместиться', 'хоккейный', 'поле', 'площадка', 'катание', 'ролик', '

Ниже представлено распараеллеливание общего датасета между процессами. На 10 000 записях данный вариант справляется за 40s, в то время как для обработки с использованием одного процесса уходит в 2 раза больше времени - 1m 18s.

In [40]:
from preprocessing import multipreprocessing_text
with multiprocessing.Pool(processes=20) as pool:
    result = pool.map(multipreprocessing_text, test_l)
print(result[0])

['ночь', 'четверг', '20', 'март', 'город', 'иваново', 'обрушиться', 'купол', 'строиться', 'спортивный', 'комплекс', 'сообщать', 'интерфакс', 'ссылка', 'главное', 'инженер', 'компанииподрядчик', 'слово', 'строитель', 'объект', 'время', 'происшествие', 'сотрудник', 'охрана', 'находиться', 'друг', 'здание', 'жертва', 'пострадать', 'купол', 'здание', 'рухнуть', 'тяжесть', 'мокрый', 'снег', 'большой', 'количество', 'выпасть', 'ночь', 'среда', 'четверг', 'ивановский', 'область', 'данные', 'синоптик', 'дорога', 'город', 'толщина', 'снег', 'достигать', '36', 'смотреть', 'сильный', 'другой', 'снегопад', 'сильный', 'шквалистый', 'ветер', 'пострадать', 'родниковский', 'ивановский', 'савинский', 'лежневский', 'район', 'область', 'строительство', 'спортивный', 'комплекс', 'должный', 'завершиться', 'маю', 'год', 'общий', 'стоимость', 'объект', 'составлять', '260', 'миллион', 'рубль', 'купол', '77', 'тысяча', 'квадратный', 'метр', 'разместиться', 'хоккейный', 'поле', 'площадка', 'катание', 'ролик', '